<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [55]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/eyvonne/csvFiles/master/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [3]:
df=df.replace(' ', np.NaN)

In [4]:
df['TotalCharges']=df.TotalCharges.astype('float')

In [5]:
df.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,2
top,5032-USPKF,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,5174


In [21]:
df=df.replace({'Yes':1, 'No':0, 'Male':0,'Female':1, 'No phone service':-1, 'No internet service':-1, })

In [27]:
df['InternetService']=df.InternetService.replace(-1,'None')

In [24]:
pd.crosstab(df['InternetService'], df['OnlineSecurity'])

OnlineSecurity,-1,0,1
InternetService,,,
-1,1526,0,0
DSL,0,1241,1180
Fiber optic,0,2257,839


In [26]:
df.describe(exclude='number')

,customerID,InternetService,Contract,PaymentMethod
count,7043,7043,7043,7043
unique,7043,3,3,4
top,5032-USPKF,Fiber optic,Month-to-month,Electronic check
freq,1,3096,3875,2365


In [41]:
cols=df.columns.tolist()[1:-1]

In [46]:
X=df[cols]
y=df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
1592,1,0,1,1,42,1,0,Fiber optic,0,0,0,0,1,0,Month-to-month,1,Credit card (automatic),77.95,3384.00
6040,1,0,0,0,1,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,0,Mailed check,70.75,70.75
2434,1,0,1,1,3,1,0,None,-1,-1,-1,-1,-1,-1,Month-to-month,0,Mailed check,19.65,60.65
6590,1,0,0,0,1,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.85,70.85
488,1,0,1,1,0,0,-1,DSL,1,0,1,1,1,0,Two year,1,Bank transfer (automatic),52.55,NaN


In [56]:
encoder=ce.OneHotEncoder(use_cat_names=True)
scaler=MinMaxScaler()
pipe=Pipeline([('encode', encoder), ('scale', scaler)])

In [65]:
pipe.fit(X_train, y_train)
X_train=pipe.transform(X_train)
X_test=pipe.transform(X_test)

/Users/eyvonnegeordan/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [69]:
def create_model():
    model=Sequential()
    model.add(Dense(12, input_dim=26, activation='sigmoid'))
    model.add(Dense(6, activation='tanh'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adagrad', metrics=['accuracy'])
    return model

In [70]:
model=KerasClassifier(build_fn=create_model, verbose=10)

In [71]:
param_grid={'batch_size':[10,20,40, 60,100], 
            'epochs':[10, 20, 100]}

grid=GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result=grid.fit(X_train, y_train)

Train on 3521 samples
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Train on 3521 samples
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Train on 3522 samples
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Train on 3521 samples
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
Train on 3521 samples
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
Train on 3522 samples
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7

In [73]:
grid_result.best_score_

0.7307837819994422

In [74]:
grid_result.best_params_

{'batch_size': 10, 'epochs': 10}

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?